In [ ]:
"""
Created on Thu Aug 21 10:32 2022

Gather cross-validation parameters for later application

@author: Clara Burgard

"""

In [ ]:
import glob
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import xarray as xr

In [ ]:
%matplotlib qt5

In [ ]:
nisf_list = [10,11,12,13,18,22,23,24,25,30,31,33,38,39,40,42,43,44,45,47,48,51,52,53,54,55,58,61,65,66,69,70,71,73,75]

In [ ]:
outputpath_simple_all = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/SIMPLE/nemo_5km_06161821_oneFRIS/'

PLUMES

In [ ]:
inputpath_CV_plumes = '/bettik/burgardc/SCRIPTS/basal_melt_param/scripts/BASH/JOB_SCRIPTS/plume_tuning_CV/'

In [ ]:
CV_type='time' # can be 'time' or 'shelves'

plume_gamma_list = []
plume_E0_list = []

for mparam in ['lazero19_2', 'lazero19_modif2']: 
    print(mparam)
    
    gamma_dom_list = []
    E0_dom_list = []
    
    for dom in [50, 1000]: #
    
        last_guess_list = []

        if CV_type == 'time':
            plume_CV_files = list(sorted(glob.glob(inputpath_CV_plumes+'plume_'+mparam+'_'+str(dom)+'km_CV_tidesFalse_noisf00_notblock*.log')))
        elif CV_type == 'shelves':
            plume_CV_files = list(sorted(glob.glob(inputpath_CV_plumes+'plume_'+mparam+'_'+str(dom)+'km_CV_tidesFalse_noisf*_notblock00.log')))

        for ffile in plume_CV_files:
            if os.stat(ffile).st_size == 0:
                print(ffile+' is empty')
            else:
                f = open(ffile, 'r')
                last_line = f.readlines()[-2]
                #print(last_line)
                if last_line.split()[0] == 'FINAL':
                    last_guess = np.array(last_line.split('=')[1].strip().split('[')[1].split(']')[0].split()).astype(float)
                    last_guess_list.append(last_guess)
                else:
                    print(ffile)
                    if mparam == 'lazero19_modif2':
                        last_guess = np.array([1000, 1.35e-3])
                        last_guess_list.append(last_guess)

        if CV_type == 'time':
            gamma_da = xr.DataArray(data=np.array(last_guess_list)[:,0], coords={'tblock': range(1,14), 'profile_domain': dom, 'param': mparam}, dims=['tblock'])
            E0_da = xr.DataArray(data=np.array(last_guess_list)[:,1], coords={'tblock': range(1,14), 'profile_domain': dom, 'param': mparam}, dims=['tblock'])
        
        elif CV_type == 'shelves':
            gamma_da = xr.DataArray(data=np.array(last_guess_list)[:,0], coords={'Nisf': nisf_list, 'profile_domain': dom, 'param': mparam}, dims=['Nisf'])
            E0_da = xr.DataArray(data=np.array(last_guess_list)[:,1], coords={'Nisf': nisf_list, 'profile_domain': dom, 'param': mparam}, dims=['Nisf'])
        
        gamma_dom_list.append(gamma_da)
        E0_dom_list.append(E0_da)
        
    gamma_dom = xr.concat(gamma_dom_list, dim='profile_domain')
    E0_dom = xr.concat(E0_dom_list, dim='profile_domain')
    
    plume_gamma_list.append(gamma_dom)
    plume_E0_list.append(E0_dom)
    
plume_gamma = xr.concat(plume_gamma_list, dim='param')
plume_E0 = xr.concat(plume_E0_list, dim='param')

plume_gamma.to_dataset(name='gamma').to_netcdf(outputpath_simple_all + 'plume_gamma_CV'+CV_type+'.nc')
plume_E0.to_dataset(name='E0').to_netcdf(outputpath_simple_all + 'plume_E0_CV'+CV_type+'.nc')

In [ ]:
plume_gamma.isel(param=0,profile_domain=0).median()

BOXES

In [ ]:
inputpath_CV_boxes = '/bettik/burgardc/SCRIPTS/basal_melt_param/scripts/BASH/JOB_SCRIPTS/box_tuning_CV/'

In [ ]:
CV_type='shelves' # can be 'time' or 'shelves'

box_gamma_list = []
box_C_list = []

pism_version = 'yes'
picop_opt = 'no'

for nD_config in range(1,5): 
    
    mparam = 'boxes_'+str(nD_config)+'_pism'+pism_version+'_picop'+picop_opt

    gamma_dom_list = []
    C_dom_list = []
    
    for dom in [50, 1000]:
    
        last_guess_list = []
        if CV_type == 'time':
            box_CV_files = list(sorted(glob.glob(inputpath_CV_boxes+'box_'+str(nD_config)+'_'+pism_version+'_'+picop_opt+'_'+str(dom)+'km_CV_tidesFalse_noisf00_notblock*.log')))
        elif CV_type == 'shelves':
            box_CV_files = list(sorted(glob.glob(inputpath_CV_boxes+'box_'+str(nD_config)+'_'+pism_version+'_'+picop_opt+'_'+str(dom)+'km_CV_tidesFalse_noisf*_notblock00.log')))

        for ffile in box_CV_files:
            if os.stat(ffile).st_size == 0:
                print(ffile+' is empty')
            else:
                f = open(ffile, 'r')
                last_line = f.readlines()[-2]
                #print(last_line)
                if last_line.split()[0] == 'FINAL':
                    last_guess = np.array(last_line.split('=')[1].strip().split('[')[1].split(']')[0].split()).astype(float)
                    last_guess_list.append(last_guess)
                else:
                    print(ffile)

        if CV_type == 'time':
            gamma_da = xr.DataArray(data=np.array(last_guess_list)[:,0], coords={'tblock': range(1,14), 'profile_domain': dom, 'param': mparam}, dims=['tblock'])
            C_da = xr.DataArray(data=np.array(last_guess_list)[:,1], coords={'tblock': range(1,14), 'profile_domain': dom, 'param': mparam}, dims=['tblock'])
        
        elif CV_type == 'shelves':
            gamma_da = xr.DataArray(data=np.array(last_guess_list)[:,0], coords={'Nisf': nisf_list, 'profile_domain': dom, 'param': mparam}, dims=['Nisf'])
            C_da = xr.DataArray(data=np.array(last_guess_list)[:,1], coords={'Nisf': nisf_list, 'profile_domain': dom, 'param': mparam}, dims=['Nisf'])
        
        gamma_dom_list.append(gamma_da)
        C_dom_list.append(C_da)
        
    gamma_dom = xr.concat(gamma_dom_list, dim='profile_domain')
    C_dom = xr.concat(C_dom_list, dim='profile_domain')
    
    box_gamma_list.append(gamma_dom)
    box_C_list.append(C_dom)
    
box_gamma = xr.concat(box_gamma_list, dim='param')
box_C = xr.concat(box_C_list, dim='param')

box_gamma.to_dataset(name='gamma').to_netcdf(outputpath_simple_all + 'box_gamma_CV'+CV_type+'.nc')
box_C.to_dataset(name='C').to_netcdf(outputpath_simple_all + 'box_C_CV'+CV_type+'.nc')

PICOP

In [ ]:
inputpath_CV_boxes = '/bettik/burgardc/SCRIPTS/basal_melt_param/scripts/BASH/JOB_SCRIPTS/box_tuning_CV/'

In [ ]:
CV_type='time' # can be 'time' or 'shelves'

picop_gamma_list = []
picop_E0_list = []



for nD_config in range(3,5): #, 'lazero19_modif2'
    
    if nD_config == 3:
        pism_version = 'yes'
        picop_opt = 'yes'
    
    elif nD_config == 4:
        pism_version = 'no'
        picop_opt = 'yes'
    
    mparam = 'boxes_'+str(nD_config)+'_pism'+pism_version+'_picop'+picop_opt

    gamma_dom_list = []
    E0_dom_list = []
    
    for dom in [50, 1000]: #
    
        last_guess_list = []
        if CV_type == 'time':
            box_CV_files = list(sorted(glob.glob(inputpath_CV_boxes+'box_'+str(nD_config)+'_'+pism_version+'_'+picop_opt+'_'+str(dom)+'km_CV_tidesFalse_noisf00_notblock*.log')))
        elif CV_type == 'shelves':
            box_CV_files = list(sorted(glob.glob(inputpath_CV_boxes+'box_'+str(nD_config)+'_'+pism_version+'_'+picop_opt+'_'+str(dom)+'km_CV_tidesFalse_noisf*_notblock00.log')))

        for ffile in box_CV_files:
            if os.stat(ffile).st_size == 0:
                print(ffile+' is empty')
            else:
                f = open(ffile, 'r')
                last_line = f.readlines()[-2]
                #print(last_line)
                if last_line.split()[0] == 'FINAL':
                    last_guess = np.array(last_line.split('=')[1].strip().split('[')[1].split(']')[0].split()).astype(float)
                    last_guess_list.append(last_guess)
                else:
                    print(ffile)

        if CV_type == 'time':
            gamma_da = xr.DataArray(data=np.array(last_guess_list)[:,0], coords={'tblock': range(1,14), 'profile_domain': dom, 'param': mparam}, dims=['tblock'])
            E0_da = xr.DataArray(data=np.array(last_guess_list)[:,1], coords={'tblock': range(1,14), 'profile_domain': dom, 'param': mparam}, dims=['tblock'])
        
        elif CV_type == 'shelves':
            gamma_da = xr.DataArray(data=np.array(last_guess_list)[:,0], coords={'Nisf': nisf_list, 'profile_domain': dom, 'param': mparam}, dims=['Nisf'])
            E0_da = xr.DataArray(data=np.array(last_guess_list)[:,1], coords={'Nisf': nisf_list, 'profile_domain': dom, 'param': mparam}, dims=['Nisf'])
        
        gamma_dom_list.append(gamma_da)
        E0_dom_list.append(E0_da)
        
    gamma_dom = xr.concat(gamma_dom_list, dim='profile_domain')
    E0_dom = xr.concat(E0_dom_list, dim='profile_domain')
    
    picop_gamma_list.append(gamma_dom)
    picop_E0_list.append(E0_dom)
    
picop_gamma = xr.concat(picop_gamma_list, dim='param')
picop_E0 = xr.concat(picop_E0_list, dim='param')

picop_gamma.to_dataset(name='gamma').to_netcdf(outputpath_simple_all + 'picop_gamma_CV'+CV_type+'.nc')
picop_E0.to_dataset(name='E0').to_netcdf(outputpath_simple_all + 'picop_E0_CV'+CV_type+'.nc')

CHECK RESULTS

In [ ]:
CV_type = 'shelves'
picop_gamma = xr.open_dataset(outputpath_simple_all + 'picop_gamma_CV'+CV_type+'.nc')['gamma']
picop_E0= xr.open_dataset(outputpath_simple_all + 'picop_E0_CV'+CV_type+'.nc')['E0']

In [ ]:
picop_gamma.sel(profile_domain=50).isel(param=1)

In [ ]:
plt.figure()
picop_E0.sel(profile_domain=50).isel(param=1).plot()
plt.ylim(0,2)

In [ ]:
CV_type = 'shelves'
box_gamma = xr.open_dataset(outputpath_simple_all + 'box_gamma_CV'+CV_type+'.nc')['gamma']
box_C= xr.open_dataset(outputpath_simple_all + 'box_C_CV'+CV_type+'.nc')['C']

In [ ]:
box_gamma.sel(profile_domain=50).isel(param=2).plot()

In [ ]:
box_C.sel(profile_domain=50).isel(param=2).plot()